In [2]:
from Bio import SeqIO
from Bio.Seq import Seq

In [3]:
rs=Seq("GGCAACTTGGCCGAGTGGTTAAGGCGAAAGATTAGAAATCTTTTGGGCTTTGCCCGCGCAGGTTCGAGTCCTGCAGTTGTCG")
len(rs)

/home/f1lem0n/.miniconda3/envs/mgr/lib/python3.10/site-packages/Bio/Seq.py:2334: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


Seq('GNLAEWLRRKIRNLLGFARAGSSPAVV')

In [3]:
with open("../data/S288C_reference_genome_R62-1-1_20090218/rna_coding_R62-1-1_20090220.fasta") as f:
    nn_rec = list(SeqIO.parse(f, "fasta"))[5]
chr = "chr" + nn_rec.description.split("Chr")[1].split(" ")[1]
start, stop = nn_rec.description.split("from ")[1].split(",")[0].split("-")
start, stop = int(start), int(stop)
if start > stop:
    start, stop = stop - 1, start
chr, start, stop

('chrI', 182515, 182597)

In [5]:
with open("../data/S288C_reference_genome_R62-1-1_20090218/S288C_reference_sequence_R62-1-1_20090218.fsa") as f:
    ref = list(SeqIO.parse(f, "fasta"))[0]
print(ref.seq[start:stop].reverse_complement() == rs)

True


In [16]:
from Bio import Entrez, SeqIO
from io import StringIO

Entrez.email = "hajdylaf@gmail.com"

In [4]:
ids = ["GHM89747.1", "CAI4583015.1"]

aa_records = []
nn_records = []

for id in ids:
    with Entrez.efetch(db="protein", id=id, rettype="gb", retmode="text") as handle:
        gb = handle.read()
    aa_rec = SeqIO.read(StringIO(gb), "genbank").format("fasta")
    aa_rec = SeqIO.read(StringIO(aa_rec), "fasta")
    aa_records.append(aa_rec)
    cds_coords = gb.split("/coded_by=")[1].split("\n")[0]
    if "(" in cds_coords:
        cds_coords = cds_coords.split("(")[1].split(")")[0]
    seqid, coords = cds_coords.split(":")[0], cds_coords.split(":")[1]
    start, stop = coords.split("..")
    start, stop = int(start), int(stop)
    if start > stop:
        start, stop = stop, start
    start -= 1
    with Entrez.efetch(db="nucleotide", id=seqid, rettype="fasta", retmode="text") as handle:
        nn_rec = list(SeqIO.parse(handle, "fasta"))[0]
    nn_rec.seq = nn_rec.seq[start:stop]
    nn_records.append(nn_rec)

SeqIO.write(aa_records, "homologs_aa.fasta", "fasta")
SeqIO.write(nn_records, "pal2nal_guides.fasta", "fasta")


2

In [9]:
with open("../data/S288C_reference_genome_R62-1-1_20090218/S288C_reference_sequence_R62-1-1_20090218.fsa") as f:
    records = list(SeqIO.parse(f, "fasta"))
edited_records = []
for rec in records:
    try:
        new_id = "chr" + rec.description.split("chromosome=")[1].split("]")[0]
    except IndexError:
        new_id = "chrMito"
    rec.id = new_id
    rec.name = new_id
    rec.description = new_id + " " + rec.description
    edited_records.append(rec)
SeqIO.write(edited_records, "../data/S288C_reference_genome_R62-1-1_20090218/S288C_reference_sequence_R62-1-1_20090218_adj.fasta", "fasta")


17

In [13]:
with open("../output/structures/mRNA/blastn-hits.csv") as f:
    lines = f.readlines()
lines = [line.strip().split(",") for line in lines if line.strip()]

In [26]:
homologs = {}
for line in lines:
    query_id = line[0]
    subject_id = line[1]
    query_start = int(line[8]) - 1
    query_stop = int(line[9])
    print("Fetching", subject_id)
    with Entrez.efetch(db="nucleotide", id=subject_id, rettype="fasta", retmode="text") as handle:
        rec = list(SeqIO.parse(handle, "fasta"))[0]
    rec.seq = rec.seq[query_start:query_stop]
    if query_id not in homologs:
        homologs[query_id] = [rec]
    else:
        homologs[query_id] += [rec]

Fetching CP133022.1
Fetching CP129569.1
Fetching CP125415.1
Fetching CP125398.1
Fetching CP080604.1
Fetching CP011669.1
Fetching CP096553.1
Fetching CP093766.1
Fetching CP093782.1
Fetching CP093750.1
Fetching CP046082.1
Fetching CP127196.1
Fetching AP027345.1
Fetching AP026833.1
Fetching CP011669.1
Fetching CP092954.1
Fetching CP089118.1
Fetching CP089101.1
Fetching CP063256.1
Fetching CP059523.2
Fetching CP127198.1
Fetching CP127198.1
Fetching CP127198.1
Fetching AP027347.1
Fetching AP027347.1
Fetching AP027347.1
Fetching AP026835.1
Fetching AP026835.1
Fetching AP026835.1
Fetching CP011671.1
Fetching CP011671.1
Fetching CP011671.1
Fetching CP072078.1
Fetching CP072078.1
Fetching CP072078.1
Fetching CP072094.1
Fetching CP072094.1
Fetching CP072094.1
Fetching CP097135.1
Fetching CP097135.1
Fetching CP097135.1
Fetching CP093816.1
Fetching CP093816.1
Fetching CP093816.1
Fetching CP093688.1
Fetching CP093688.1
Fetching CP093688.1
Fetching CP093672.1
Fetching CP093672.1
Fetching CP093672.1


In [27]:
for query_id in homologs:
    with open(f"../output/structures/mRNA/{query_id}/seqdump.txt", "w") as f:
        SeqIO.write(homologs[query_id], f, "fasta")

In [28]:
with open("../output/structures/mRNA/blastp-hits.csv") as f:
    lines = f.readlines()
lines = [line.strip().split(",") for line in lines if line.strip()]

In [34]:
for line in lines:
    query_id = line[0]
    to_save = [line for line in lines if line[0] == query_id]
    with open(f"../output/structures/mRNA/{query_id}/hits.csv", "w") as f:
        f.write("\n".join([",".join(line) for line in to_save]))

In [1]:
from pathlib import Path
import pandas as pd
root = Path("../output/structures/")
df = pd.DataFrame()
for type in ["mRNA", "tRNA", "rRNA"]:
    for file in (root / type).iterdir():
        if file.suffix == ".tsv":
            _df = pd.read_csv(file, sep="\t")
            _df = _df.melt(id_vars=["gene"],
                         var_name="structure",
                         value_name="score")
            _df["type"] = type
            df = pd.concat([df, _df])
        if file.is_file():
            continue
df["IA"] = 0.0

for type in ["mRNA", "tRNA", "rRNA"]:
    for file in (root / type).iterdir():
        if file.is_file():
            continue
        with open(file / "constrained.fasta") as f:
            lines = f.readlines()
        seq = lines[1].strip()
        con = lines[2].strip()
        ia = con.count("x") / len(seq)
        df.loc[df["gene"] == file.name, "IA"] = ia


In [2]:
df.to_csv("../output/structures/combined.csv", index=False)